In [3]:
from keras.models import Model
from keras.layers import Input, Conv2D, Activation, BatchNormalization, Dropout, concatenate, Conv2DTranspose
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [4]:
input_color = Input(shape=(256,256,3), name="Color_Input")
input_ir = Input(shape=(256,256,1), name="IR_Input")

# Color branch
x = Conv2D(256, (5,5), strides=(2,2), padding="same", activation="relu", name="Color_Conv2D_1")(input_color)
x = Conv2D(128, (3,3), strides=(2,2), padding="same", activation="relu", name="Color_Conv2D_2")(x)
x = Model(inputs=input_color, outputs=x)

# IR branch
y = Conv2D(256, (5,5), strides=(2,2), padding="same", activation="relu", name="IR_Conv2D_1")(input_ir)
y = Conv2D(128, (3,3), strides=(2,2), padding="same", activation="relu", name="IR_Conv2D_2")(y)
y = Model(inputs=input_ir, outputs=y)

# combine both branches
combined = concatenate([x.output, y.output], name="Concat")

z = Conv2D(64, (3,3), strides=(1,1), padding="same", activation="relu", name="Comb_Conv2D_1")(combined)
z = Conv2DTranspose(128, kernel_size=(3,3), strides=(2,2), padding="same", activation="relu", name="Comb_DeConv2D_1")(z)
z = Conv2DTranspose(256, kernel_size=(5,5), strides=(2,2), padding="same", activation="relu", name="Comb_DeConv2D_2")(z)
z = Conv2D(256, kernel_size=(5,5), strides=(1,1), padding="same", activation="relu", name="Comb_Conv2D_2")(z)
z = Conv2D(1, kernel_size=(3,3), strides=(1,1), padding="same", activation="relu", name="Comb_Conv2D_3")(z)

model = Model(inputs=[x.input, y.input], outputs=z)

model.compile(
    optimizer="adam",
    loss="mae",
    metrics=['mae', 'mse'])

W0913 15:17:51.308677 140074136270656 deprecation_wrapper.py:119] From /home/julien/anaconda3/envs/mlenv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0913 15:17:51.323953 140074136270656 deprecation_wrapper.py:119] From /home/julien/anaconda3/envs/mlenv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0913 15:17:51.356820 140074136270656 deprecation_wrapper.py:119] From /home/julien/anaconda3/envs/mlenv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0913 15:17:51.466665 140074136270656 deprecation_wrapper.py:119] From /home/julien/anaconda3/envs/mlenv/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.

In [5]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Color_Input (InputLayer)        (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
IR_Input (InputLayer)           (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
Color_Conv2D_1 (Conv2D)         (None, 128, 128, 256 19456       Color_Input[0][0]                
__________________________________________________________________________________________________
IR_Conv2D_1 (Conv2D)            (None, 128, 128, 256 6656        IR_Input[0][0]                   
__________________________________________________________________________________________________
Color_Conv